In [1]:
#    1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
#    2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
#    3. month - month of the year: "jan" to "dec" 
#    4. day - day of the week: "mon" to "sun"
#    5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
#    6. DMC - DMC index from the FWI system: 1.1 to 291.3 
#    7. DC - DC index from the FWI system: 7.9 to 860.6 
#    8. ISI - ISI index from the FWI system: 0.0 to 56.10
#    9. temp - temperature in Celsius degrees: 2.2 to 33.30
#    10. RH - relative humidity in %: 15.0 to 100
#    11. wind - wind speed in km/h: 0.40 to 9.40 
#    12. rain - outside rain in mm/m2 : 0.0 to 6.4 
#    13. area - the burned area of the forest (in ha): 0.00 to 1090.84 
#    (this output variable is very skewed towards 0.0, thus it may make
#     sense to model with the logarithm transform). 

In [13]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [14]:
# Load data

UCI_dataset = np.loadtxt("forestfires.csv", 
                         dtype= 'str', skiprows=1, delimiter=',')

X = UCI_dataset[:,0:12]
Y = UCI_dataset[:,12]
print(X.shape,Y.shape)

(517, 12) (517,)


In [16]:
# Shuffles the data before split
# Train/Test split
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size = 0.3)
print("# Train: {} , #Test: {}".format(X_train.shape[0], X_test.shape[0]))
print("# inputs: {}".format(X_train.shape[1]))
n = X_train.shape[1]

# Train: 361 , #Test: 156
# inputs: 12
